# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
# print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

all_files = []

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    for f in file_path_list :
        all_files.append(os.path.abspath(f))
    
# print(file_path_list)
# print(len(file_path_list))

print(len(all_files))
# print(all_files)

31


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in all_files:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8071


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6832


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity 
        WITH REPLICATION = 
        {'class': 'SimpleStrategy', 'replication_factor': 1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS music_library"
query = query + "(artist_name text, song_title text, song_length text, sessionId int, itemInSession int, \
 PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [9]:
import pdb

In [10]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library (artist_name, song_title, song_length, sessionId, itemInSession)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
#         pdb.set_trace()
        session.execute(query, (line[0], line[9], line[5], int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "select * from music_library limit 10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(sessionid=23, iteminsession=0, artist_name='Regina Spektor', song_length='191.08526', song_title='The Calculation (Album Version)')
Row(sessionid=23, iteminsession=1, artist_name='Octopus Project', song_length='250.95791', song_title='All Of The Champs That Ever Lived')
Row(sessionid=23, iteminsession=2, artist_name='Tegan And Sara', song_length='180.06159', song_title='So Jealous')
Row(sessionid=23, iteminsession=3, artist_name='Dragonette', song_length='153.39057', song_title='Okay Dolores')
Row(sessionid=23, iteminsession=4, artist_name='Lil Wayne / Eminem', song_length='229.58975', song_title='Drop The World')
Row(sessionid=23, iteminsession=5, artist_name="Soulja Boy Tell'em", song_length='201.11628', song_title='Let Me Get Em')
Row(sessionid=23, iteminsession=6, artist_name='Bodo Wartke', song_length='645.27628', song_title='Liebeslied (Sprachen: Deutsch_ Englisch_ FranzÃ\x83Â¶sisch_ Italienisch_ Spanisch_ HollÃ\x83Â¤ndisch_ Japanisch_ Russisch_ Griechisch_ Klingonisch_ Hessi

In [12]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "SELECT artist_name, song_title, song_length FROM music_library WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length='495.3073')


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

#Create Table based on Query

query = "CREATE TABLE IF NOT EXISTS music_library1"
query = query + "(artist_name text, song_title text, first_name text, last_name text,itemInSession int, \
 userid int, sessionid int,PRIMARY KEY (userid, sessionid, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library1 (artist_name, song_title, first_name, last_name, itemInSession, \
        userid, sessionid)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
#         pdb.set_trace()
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[3]), int(line[10]), int(line[8])))

In [15]:
query = "SELECT artist_name,song_title,first_name,last_name FROM music_library1 WHERE userid = 10 AND sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


In [16]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#Create Table based on Query

query = "CREATE TABLE IF NOT EXISTS music_library2"
query = query + "(first_name text, last_name text, userid int, song_title text, PRIMARY KEY (song_title, userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)                   

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library2 (first_name, last_name, userid, song_title)"
        query = query + "VALUES(%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
#         pdb.set_trace()
        session.execute(query, (line[1], line[4], int(line[10]), line[9]))

In [18]:
query = "SELECT first_name,last_name FROM music_library2 WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


### Drop the tables before closing out the sessions

In [19]:
## TO-DO: Drop the table before closing out the sessions
query = "drop table music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
    
query = "drop table music_library1"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table music_library2"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()